<a href="https://colab.research.google.com/github/da-shinjk/dlworkshop_02/blob/master/dlworkshop_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

第1回で入力したコード。

この環境で動作しているPythonバージョンを調べます。

In [0]:
import sys
print(sys.version)

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


以下からが第2回の内容です。

まず、あやめの特徴を示したデータセットをロードします。

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()

データセットに含まれているデータを5つ表示してみます。

In [2]:
for idx, item in enumerate(zip(iris.data, iris.target)):
    if idx == 5:
        break
    print('data:', item[0], ', target:', item[1])

data: [5.1 3.5 1.4 0.2] , target: 0
data: [4.9 3.  1.4 0.2] , target: 0
data: [4.7 3.2 1.3 0.2] , target: 0
data: [4.6 3.1 1.5 0.2] , target: 0
data: [5.  3.6 1.4 0.2] , target: 0


データセットを訓練データとテストに分割します。

In [13]:
from sklearn.model_selection import train_test_split
print('length of iris.data:', len(iris.data))  # iris.dataのデータ数
print('length of iris.target:', len(iris.target))  # iris.targetのデータ数

# iris.dataとiris.targetに含まれるデータをシャッフルして分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)
print('length of X_train:', len(X_train))
print('length of y_train:', len(y_train))
print('length of X_test:', len(X_test))
print('length of y_test:', len(y_test))

length of iris.data: 150
length of iris.target: 150
length of X_train: 112
length of y_train: 112
length of X_test: 38
length of y_test: 38


分割後のデータセット（X_trainとy_train）のデータを5つ表示します。

In [14]:
for idx, item in enumerate(zip(X_train, y_train)):
    if idx == 5:
        break
    print('data:', item[0], ', target:', item[1])

data: [5.6 2.5 3.9 1.1] , target: 1
data: [5.9 3.  4.2 1.5] , target: 1
data: [6.5 3.  5.2 2. ] , target: 2
data: [5.2 3.5 1.5 0.2] , target: 0
data: [6.8 3.2 5.9 2.3] , target: 2


データをPyTorchで使えるように変換します。

In [0]:
import torch

X_train = torch.from_numpy(X_train).float()
y_train = torch.tensor([[float(x)] for x in y_train])
X_test = torch.from_numpy(X_test).float()
y_test = torch.tensor([[float(x)] for x in y_test])

ニューラルネットワークを表すクラスを定義します。

In [0]:
from torch import nn

INPUT_FEATURES = 4
HIDDEN = 5
OUTPUT_FEATURES = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(INPUT_FEATURES, HIDDEN)
        self.fc2 = nn.Linear(HIDDEN, OUTPUT_FEATURES)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        return x

定義したニューラルネットワークを使ってみます。

In [17]:
net = Net()  # ニューラルネットワークのインスタンスを生成

outputs = net(X_train[0:3])  # 訓練データの先頭から3個の要素を入力
print(outputs)
for idx in range(3):
    print('output:', outputs[idx], ', label:', y_train[idx])

tensor([[0.5395],
        [0.5385],
        [0.5209]], grad_fn=<AddmmBackward>)
output: tensor([0.5395], grad_fn=<SelectBackward>) , label: tensor([1.])
output: tensor([0.5385], grad_fn=<SelectBackward>) , label: tensor([1.])
output: tensor([0.5209], grad_fn=<SelectBackward>) , label: tensor([2.])


先ほど定義したニューラルネットワークを使って、学習を行います。

In [18]:
net = Net()  # ニューラルネットワークのインスタンスを生成

criterion = nn.MSELoss()  # 損失関数
optimizer = torch.optim.SGD(net.parameters(), lr=0.003)  # 最適化手法

EPOCHS = 2000  # 上と同じことを2000回繰り返す
for epoch in range(EPOCHS):
    optimizer.zero_grad()  # 重みとバイアスの更新で内部的に使用するデータをリセット
    outputs = net(X_train)  # 手順1：ニューラルネットワークにデータを入力
    loss = criterion(outputs, y_train)  # 手順2：正解ラベルとの比較
    loss.backward()  # 手順3-1：誤差逆伝播
    optimizer.step()  # 手順3-2：重みとバイアスの更新
    
    if epoch % 100 == 99:  # 100回繰り返すたびに損失を表示
        print(f'epoch: {epoch+1:4}, loss: {loss.data}')

print('training finished')

epoch:  100, loss: 0.6393837928771973
epoch:  200, loss: 0.5842536687850952
epoch:  300, loss: 0.5612550973892212
epoch:  400, loss: 0.5348739624023438
epoch:  500, loss: 0.5019711852073669
epoch:  600, loss: 0.4604303240776062
epoch:  700, loss: 0.40949442982673645
epoch:  800, loss: 0.35218316316604614
epoch:  900, loss: 0.29507753252983093
epoch: 1000, loss: 0.24343958497047424
epoch: 1100, loss: 0.19943009316921234
epoch: 1200, loss: 0.16361494362354279
epoch: 1300, loss: 0.13570287823677063
epoch: 1400, loss: 0.1147782951593399
epoch: 1500, loss: 0.09958261251449585
epoch: 1600, loss: 0.08879823237657547
epoch: 1700, loss: 0.08124153316020966
epoch: 1800, loss: 0.0759531706571579
epoch: 1900, loss: 0.07221029698848724
epoch: 2000, loss: 0.0694972425699234
training finished


最後の学習での計算結果と対応する正解ラベルを表示してみます。

In [19]:
for idx, item in enumerate(zip(outputs, y_train)):
    if idx == 5:
        break
    print(item[0].data, '<--->', item[1])

tensor([1.1532]) <---> tensor([1.])
tensor([1.2502]) <---> tensor([1.])
tensor([1.5774]) <---> tensor([2.])
tensor([0.0143]) <---> tensor([0.])
tensor([1.7571]) <---> tensor([2.])


計算結果を整数値に補正するコード。

In [20]:
predict = (outputs + 0.5).int()
for idx, item in enumerate(zip(predict, y_train)):
    if idx == 5:
        break
    print('output:', item[0], ', label:', item[1])

output: tensor([1], dtype=torch.int32) , label: tensor([1.])
output: tensor([1], dtype=torch.int32) , label: tensor([1.])
output: tensor([2], dtype=torch.int32) , label: tensor([2.])
output: tensor([0], dtype=torch.int32) , label: tensor([0.])
output: tensor([2], dtype=torch.int32) , label: tensor([2.])


入力されたデータから推測された結果が、正解ラベルと一致した数を計算します。

In [22]:
compare = predict == y_train
print(compare[0:5])
print(compare.sum())

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
tensor(108)


上で学習を行ったニューラルネットワークに、テストデータを入力して、その精度を確認します。

In [23]:
outputs = net(X_test)

predict = (outputs + 0.5).int()
compare = predict == y_test

print(f'correct: {compare.sum()} / {len(predict)}')
for value, label in zip(predict, y_test):
    print('predicted:', iris.target_names[value.item()], '<--->',
        'label:', iris.target_names[int(label.item())])

correct: 38 / 38
predicted: versicolor <---> label: versicolor
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: versicolor <---> label: versicolor
predicted: versicolor <---> label: versicolor
predicted: versicolor <---> label: versicolor
predicted: virginica <---> label: virginica
predicted: setosa <---> label: setosa
predicted: setosa <---> label: setosa
predicted: versicolor <---> label: versicolor
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: virginica <---> label: virginica
predicted: versicolor <---> label: versicolor
predicted: versicolor <---> label: versicolor
predicted: virginica <---> label: virginica
predicted: setosa <---> label: setosa
predicted: virginica <---> label: virginica
predicted: virginica <---> labe